In [1]:
%load_ext autoreload
%autoreload 2
from llamawrapper import LlamaHelper

In [2]:
llama = LlamaHelper(dir='/dlabscratch1/public/llm_weights/llama2_hf/Llama-2-7b-hf', load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
class Llama2Embedding():
    def __init__(self, model, 
                 template: str = "(*):<word> (*):<word> (*):",
                 keyword: str = "<word>", 
                 layer_idx: int = 20):
        """
        model: LlamaWrapper
        """
        self.model = model
        self.template = template 
        self.keyword = keyword
        self.layer_idx = layer_idx
    
    def encode(self, sentences, batch_size=4, **kwargs):
        """
        Returns a list of embeddings for the given sentences.
        Args:
            sentences (`List[str]`): List of sentences to encode
            batch_size (`int`): Batch size for the encoding

        Returns:
            `List[np.ndarray]` or `List[tensor]`: List of embeddings for the given sentences
        """
        batch_size = 2
        prompts = [self.template.replace(self.keyword, sentence) for sentence in sentences]
        batch = []
        embs = []
        for prompt in prompts:
            batch += [prompt]
            if len(batch) == batch_size:
                latents = self.model.latents_all_layers(batch).cpu().detach()[self.layer_idx, :, -1].float()
                print(latents.shape)
                embs += [latent for latent in latents]
                batch = []
                self.model.reset_all()
        if len(batch) > 0:
            latents = self.model.latents_all_layers(batch).cpu().detach()[self.layer_idx, :, -1].float()
            print(latents.shape)
            embs += [latent for latent in latents]
            self.model.reset_all()
        return embs


In [12]:
llama.reset_all()
model = Llama2Embedding(llama)

embs = model.encode(["dog", "cat", "house", "day", "night"], 2)

['dog', 'cat', 'house', 'day', 'night']
2
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([1, 4096])


In [13]:
print(len(embs), embs[0].shape)

5 torch.Size([4096])


In [16]:
from mteb import MTEB
evaluation = MTEB(tasks=["EmotionClassification"])
evaluation.run(model)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- EmotionClassification, s2s

/dlabdata1/wendler/.pt201/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Error while evaluating EmotionClassification: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).


['ive made it through a week i just feel beaten down', 'i feel this strategy is worthwhile', 'i feel so worthless and weak what does he have to say that s what i want to find out', 'i feel clever nov', 'im moved in ive been feeling kind of gloomy', 'i allowed myself to feel the really shitty feelings while i was running because a the endorphins were flowing so it hurt less and b so i could pretend i was running away from them', 'i feel confused too', 'i feel like a crappy mummy if were stuck in but there are days where i really cant face much else then venturing out to the garden at pm', 'i feel like i liked my hair much better before i was using a sulfate free brand and i believe i am using a reputable brand', 'i feel the self pressured expectation to keep up to date with our family events so in order to assuage the guilt here we go', 'i get it she feel betrayed and hurt', 'i cant shake off my feelings of being offended and hurt no matter how hard i try and the conversation keeps cons

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).